<a href="https://colab.research.google.com/github/arbouria/Notas-Aprendizaje-y-Comportamiento-Adaptable-I/blob/main/Simulador_Interactivo_en_Python_del_Modelo_de_Baum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Simulador interactivo del Modelo de Baum

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import ipywidgets as widgets
from ipywidgets import interactive, Layout
from IPython.display import display

# --- Definición de las Funciones del Modelo de Baum ---
# Basado en el artículo "Explaining Performance on Interval and Ratio Schedules
# with a Molar View of Behavior" por William M. Baum (2025)

def feedback_vr(B, vr_value):
    """
    Calcula la tasa de PIE (R) para una tasa de actividad (B) en un programa VR.
    """
    if vr_value <= 0:
        return np.inf
    return B / vr_value

def feedback_vi(B, vi_value_minutes, a=1.0):
    """
    Calcula la tasa de PIE (R) para una tasa de actividad (B) en un programa VI.
    """
    if B <= 0:
        return 0
    t_seconds = vi_value_minutes * 60
    return B / ((t_seconds / 60) * B + a)

def calculate_Vn(V, V0, K, t_vn=3.0):
    """
    Calcula el peso competitivo de las actividades no relacionadas con el PIE (VN).
    """
    Bn_numerator = K - (V + V0)
    Bn_denominator = t_vn * (V + V0) + 1

    if Bn_denominator == 0 or Bn_numerator < 0:
        return np.inf

    Bn = Bn_numerator / Bn_denominator
    if Bn <= 0:
        return np.inf

    return 1 / (t_vn + 1 / Bn)

def calculate_B_from_R_star(R_star, params):
    """
    Calcula la tasa de actividad inducida (B) para una tasa de PIE requerida (R*).
    """
    K, c1, c0, S1, S0 = params['K'], params['c1'], params['c0'], params['S1'], params['S0']

    if R_star <= 0:
        return 0

    V = c1 * (R_star ** S1)
    V0 = c0 * (R_star ** S0)
    Vn = calculate_Vn(V, V0, K)

    denominator = V + V0 + Vn
    if denominator <= 0 or np.isnan(denominator):
        return 0

    return (K * V) / denominator

def find_equilibrium(schedule_type, schedule_value, params):
    """
    Encuentra el punto de equilibrio (B, R) para un programa y parámetros dados.
    """
    def difference_function(B):
        if B < 0 or np.isnan(B): return np.inf

        R_feedback = feedback_vr(B, schedule_value) if schedule_type == 'VR' else feedback_vi(B, schedule_value)
        if R_feedback <= 0 or np.isnan(R_feedback): return -B

        def r_star_diff(R_star):
            if np.isnan(R_star) or R_star < 0: return np.inf
            return calculate_B_from_R_star(R_star, params) - B

        try:
            R_star_solution = fsolve(r_star_diff, x0=R_feedback, xtol=1e-6)[0]
        except:
            return np.inf

        if np.isnan(R_star_solution): return np.inf

        return R_feedback - (R_star_solution if R_star_solution >= 0 else 0)

    try:
        B_equilibrium = fsolve(difference_function, x0=params['K'] / 2, xtol=1e-6)[0]
    except:
        B_equilibrium = 0

    if B_equilibrium < 0.1 or np.isnan(B_equilibrium):
        B_equilibrium = 0

    R_equilibrium = feedback_vr(B_equilibrium, schedule_value) if schedule_type == 'VR' else feedback_vi(B_equilibrium, schedule_value)

    return B_equilibrium, R_equilibrium if B_equilibrium > 0 and not np.isnan(R_equilibrium) else 0

# --- Función Principal para la Simulación Interactiva ---

def run_interactive_simulation(K_vr, K_vi, c1, c0, S1, S0):
    params_vr = {'K': K_vr, 'c1': c1, 'c0': c0, 'S1': S1, 'S0': S0}
    params_vi = {'K': K_vi, 'c1': c1, 'c0': c0, 'S1': S1, 'S0': S0}

    vr_schedules = [640, 320, 160, 80, 40, 20, 10]
    vi_schedules_min = [16, 8, 4, 2, 1, 0.5]

    results_vr = [find_equilibrium('VR', vr, params_vr) for vr in vr_schedules]
    results_vi = [find_equilibrium('VI', vi, params_vi) for vi in vi_schedules_min]

    vr_B, vr_R = zip(*[(b, r) for b, r in results_vr if b > 0])
    vi_B, vi_R = zip(*[(b, r) for b, r in results_vi if b > 0])

    # Extraer el punto de umbral para VR 320
    vr_320_res = find_equilibrium('VR', 320, params_vr)

    # --- Graficar Resultados ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
    fig.suptitle("Simulación Interactiva del Modelo de Baum", fontsize=16)

    # Gráfica 1: Coordenadas Logarítmicas
    ax1.plot(vr_R, vr_B, 'o--', label='VR (simulado)')
    ax1.plot(vi_R, vi_B, 's-', label='VI (simulado)')
    if vr_320_res[0] > 0:
        ax1.plot(vr_320_res[1], vr_320_res[0], 'x', markersize=10, color='red', label='VR 320 (Umbral)')
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_title('Gráfica en Coordenadas Logarítmicas')
    ax1.set_xlabel('Tasa de PIE / minuto (log)')
    ax1.set_ylabel('Actividad / minuto (log)')
    ax1.legend()
    ax1.grid(True, which="both", ls="--", linewidth=0.5)

    # Gráfica 2: Coordenadas Aritméticas
    ax2.plot(vr_R, vr_B, 'o--', label='VR (simulado)')
    ax2.plot(vi_R, vi_B, 's-', label='VI (simulado)')
    if vr_320_res[0] > 0:
        ax2.plot(vr_320_res[1], vr_320_res[0], 'x', markersize=10, color='red', label='VR 320 (Umbral)')
    ax2.set_title('Gráfica en Coordenadas Aritméticas')
    ax2.set_xlabel('Tasa de PIE / minuto')
    ax2.set_ylabel('Actividad / minuto')
    ax2.legend()
    ax2.grid(True, ls="--", linewidth=0.5)
    ax2.set_ylim(bottom=0)
    ax2.set_xlim(left=0)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# --- Creación de los Widgets Interactivos ---
style = {'description_width': 'initial'}
layout = Layout(width='500px')

w = interactive(run_interactive_simulation,
                K_vr=widgets.IntSlider(min=50, max=400, step=10, value=200, description='K (VR) - Tasa Máxima', style=style, layout=layout),
                K_vi=widgets.IntSlider(min=50, max=400, step=10, value=100, description='K (VI) - Tasa Máxima', style=style, layout=layout),
                c1=widgets.FloatSlider(min=1, max=20, step=0.5, value=10, description='c1 - Coef. Conducta Operante', style=style, layout=layout),
                c0=widgets.FloatSlider(min=1, max=20, step=0.5, value=5, description='c0 - Coef. Otra Conducta', style=style, layout=layout),
                S1=widgets.FloatSlider(min=0.1, max=2, step=0.01, value=0.85, description='S1 - Exp. Sensibilidad Operante', style=style, layout=layout),
                S0=widgets.FloatSlider(min=0.01, max=2, step=0.01, value=0.05, description='S0 - Exp. Sensibilidad Otra', style=style, layout=layout)
               )

# Mostrar los widgets y la salida del gráfico
display(w)

interactive(children=(IntSlider(value=200, description='K (VR) - Tasa Máxima', layout=Layout(width='500px'), m…